In [1]:
import logging
import os
import gzip

import pandas
import easydict

import starapi.analysis
import starapi.conf

In [2]:
starapi.analysis.logger.propagate = False
starapi.conf.configure('data')

In [3]:
query_df = pandas.read_table('data/queries.tsv')
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Nonsmoker == 'Nonsmoker'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC' or PC_tissue_case == 'PC_tissue_case',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
errors = list()

for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = os.path.join('data', 'doslim', params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler(os.path.join(directory, 'log.txt'))
    logfile.setLevel(logging.DEBUG)
    starapi.analysis.logger.addHandler(logfile)
    try:
        sample_df, fold_df, balanced_perm_df, perm_df = starapi.analysis.perform_analysis(params)
        for df in fold_df, balanced_perm_df:
            if df is not None:
                df.reset_index(inplace=True)
        
    except Exception as e:
        sample_df, fold_df, balanced_perm_df, perm_df = None, None, None, None
        errors.append({'slim_id': row.slim_id, 'name': name, 'error': e})
        print(e)
    starapi.analysis.logger.removeHandler(logfile)
    
    # write files
    files = [
        (sample_df, 'samples.tsv', open),
        (fold_df, 'fold_change.tsv.gz', gzip.open),
        (balanced_perm_df, 'balanced_permutation.tsv.gz', gzip.open),
    ]
    for df, filename, opener in files:
        if df is None:
            continue
        path = os.path.join(directory, filename)
        with opener(path, 'wt') as write_file:
            df.to_csv(write_file, index=False, sep='\t', float_format='%.5g')


Started DOID_0050156 analysis
/home/dhimmels/anaconda3/envs/stargeo/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
333.75 ms in get_analysis_df("IPF == 'IPF'", "IPF_control == 'IPF_c..., '')
334.60 ms in Loading dataframe for DOID_0050156
Matching sources: 12
Stats: 12 sources, 12 series, 8 platforms, 679 samples
Loading data and calculating fold change for DOID_0050156
Loading data for GSE10667, id = 179
  1.14 s in get_probes(10)
  4.69 s in load_gse( series_id platform_id..., 179, False)
/home/dhimmels/Documents/serg/rephetio/construct/star_api/starapi/analysis.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


0 DOID:0050156 idiopathic pulmonary fibrosis
1

Started DOID_0050741 analysis
317.48 ms in get_analysis_df("alcoholism == 'alcoho..., "alcoholism_control ==..., '')
318.35 ms in Loading dataframe for DOID_0050741
Matching sources: 8
Stats: 8 sources, 8 series, 8 platforms, 435 samples
Loading data and calculating fold change for DOID_0050741
Loading data for GSE10356, id = 95
 31.17 ms in get_probes(36)
  3.37 s in load_gse( series_id platform_id..., 95, False)
 39.07 ms in get_gene_fold_change(<Gse GSE10356>, False, 0, None)
  3.44 s in Load/fold for DOID_0050741
  3.79 s in perform_analysis({'analysis_name': 'DOI...)


 DOID:0050741 alcohol dependence
zero-size array to reduction operation maximum which has no identity

Started DOID_0050742 analysis
423.09 ms in get_analysis_df("Smoker == 'Smoker'", "Nonsmoker == 'Nonsmok..., '')
423.96 ms in Loading dataframe for DOID_0050742
Matching sources: 21
Excluded 2 sources as single-class
Stats: 19 sources, 17 series, 6 platforms, 1423 samples
Loading data and calculating fold change for DOID_0050742
Loading data for GSE10006, id = 4
  1.16 s in get_probes(4)
  8.62 s in load_gse( series_id platform_id..., 4, False)
  1.32 s in get_gene_fold_change(<Gse GSE10006>, False, 0, None)
Loading data for GSE11784, id = 450
  1.28 s in get_probes(4)
 14.38 s in load_gse( series_id platform_id..., 450, False)
  2.21 s in get_gene_fold_change(<Gse GSE11784>, False, 0, None)
Loading data for GSE11906, id = 487
  1.33 s in get_probes(4)
 19.17 s in load_gse( series_id platform_id..., 487, False)
  3.00 s in get_gene_fold_change(<Gse GSE11906>, False, 0, None)
Loading data for GSE19407, id = 2430
  1.27 s in get_probes(4)
 11.86 s in load_gse( series_id platform_id..., 24

...

 DOID:9970 obesity
Loading URL ftp://ftp.ncbi.nih.gov/pub/geo/DATA/SeriesMatrix/GSE60403/GSE60403_series_matrix.txt.gz ...
Cache to

  1.38 s in get_probes(4)
  7.96 s in load_gse( series_id platform_id..., 10836, False)
484.48 ms in get_gene_fold_change(<Gse GSE60403>, False, 0, None)
Loading data for GSE9624, id = 11780


 data/geo_mirror/DATA/SeriesMatrix/GSE60403/GSE60403_series_matrix.txt.gz
Loading URL ftp://ftp.ncbi.nih.gov/pub/geo/DATA/SeriesMatrix/GSE9624/GSE9624_series_matrix.txt.gz ...
Cache to

  1.22 s in get_probes(4)
  5.88 s in load_gse( series_id platform_id..., 11780, False)
408.65 ms in get_gene_fold_change(<Gse GSE9624>, False, 0, None)
 63.30 s in Load/fold for DOID_9970
Meta-Analyzing DOID_9970
 63.63 s in meta analysis of real data for DOID_9970
269.45 ms in meta analysis of permutations for DOID_9970
 20.03 mks in get_balanced_permutations({'analysis_name': 'DOI...,  C H H_lower H_upper \..., Empty DataFrame Column...)
 63.99 s in Meta analysis for DOID_9970
DONE DOID_9970 analysis
127.78 s in perform_analysis({'analysis_name': 'DOI...)


 data/geo_mirror/DATA/SeriesMatrix/GSE9624/GSE9624_series_matrix.txt.gz


In [6]:
error_df = pandas.DataFrame(errors)
error_df = query_df.merge(error_df)
error_df

,slim_id,slim_name,case_query,control_query,error,name
0,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control',zero-size array to reduction operation maximum...,DOID_0050741
1,DOID:10283,prostate cancer,PC == 'PC' or PC_tissue_case == 'PC_tissue_case',PC_control == 'PC_control' or PC_tissue_contro...,zero-size array to reduction operation maximum...,DOID_10283
2,DOID:11054,urinary bladder cancer,bladder_cancer == 'bladder_cancer',bladder_cancer_control == 'bladder_cancer_cont...,'Int64Index' object has no attribute 'labels',DOID_11054
3,DOID:11714,gestational diabetes,GDM == 'GDM',GDM_control == 'GDM_control',zero-size array to reduction operation maximum...,DOID_11714
4,DOID:12849,autistic disorder,autism == 'autism',autism_control == 'autism_control',zero-size array to reduction operation maximum...,DOID_12849
5,DOID:14221,metabolic syndrome X,MetS == 'MetS',MetS_Control == 'MetS_Control','Int64Index' object has no attribute 'labels',DOID_14221
6,DOID:1686,glaucoma,glaucoma == 'glaucoma',glaucoma_control == 'glaucoma_control',zero-size array to reduction operation maximum...,DOID_1686
7,DOID:2377,multiple sclerosis,MS == 'MS',MS_control == 'MS_control',zero-size array to reduction operation maximum...,DOID_2377
8,DOID:2841,asthma,asthma == 'asthma',asthma_control == 'asthma_control',zero-size array to reduction operation maximum...,DOID_2841
9,DOID:2998,testicular cancer,testicular_cancer == 'testicular_cancer',testicular_cancer_control == 'testicular_cance...,'Int64Index' object has no attribute 'labels',DOID_2998


In [7]:
error_df.to_csv('data/errors.csv', index=False)